<a href="https://colab.research.google.com/github/rahiakela/math-for-programmers/blob/main/3-ascending-to-3d-world/5_rendering_3d_object_in_2d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Rendering a 3D object in 2D

Let’s try using what we’ve learned to render a simple
3D shape called an octahedron. Whereas a cube has
six faces, all of which are squares, an octahedron has
eight faces, all of which are triangles. You can think of
an octahedron as two, four-sided pyramids stacked on
top of each other.

<img src='https://github.com/rahiakela/img-repo/blob/master/math-for-programmers/octahedron.png?raw=1' width='200'/>

If this were a solid, we wouldn’t be able to see the
opposite sides.

Rendering the octahedron comes down to identifying the four triangles we need to show and shading them appropriately.

## Setup

In [ ]:
%%shell

wget https://raw.githubusercontent.com/rahiakela/math-for-programmers/main/3-ascending-to-3d-world/vectors.py
wget https://raw.githubusercontent.com/rahiakela/math-for-programmers/main/3-ascending-to-3d-world/colors.py
wget https://raw.githubusercontent.com/rahiakela/math-for-programmers/main/3-ascending-to-3d-world/draw2d.py
wget https://raw.githubusercontent.com/rahiakela/math-for-programmers/main/3-ascending-to-3d-world/draw3d.py

In [2]:
from draw3d import *
from vectors import *

%load_ext autoreload
%autoreload 2

## Defining a 3D object with vectors

An octahedron is an easy example because it has only
six corners or vertices. We can give them simple coordinates:
`(1, 0, 0), (0, 1, 0), (0, 0, 1)` and their three
opposite vectors.

<img src='https://github.com/rahiakela/img-repo/blob/master/math-for-programmers/octahedron-vertices.png?raw=1' width='800'/>

These six vectors define the boundaries of the
shape but don’t provide all the information we need
to draw it. We also need to decide which of these vertices
connect to form edges of the shape.

For instance, the top point is (0, 0, 1) and it connects
by an edge to all four points in the x,y plane.

These edges outline the top pyramid of the octahedron. Note that there is no edge from (0, 0, 1) to (0, 0, –1) because that segment would lie within the octahedron, not on its outside. Each edge is defined by a pair of vectors: the start and end points of the edge as a line segment. For instance, (0, 0, 1) and (1, 0, 0) define one of the edges.

Edges still aren’t enough data to complete the drawing. We also need to know
which triples of vertices and edges define the triangular faces we want to fill with a solid, shaded color. Here’s where orientation comes in: we want to know not only which segments define faces of the octahedron, but also whether they face toward us or away from us.

Here’s the strategy: we’ll model a triangular face as three vectors v1, v2, and v3, defining its edges. (Note that here I use subscripts 1, 2, and 3 to distinguish three different vectors, not components of the same vector.) Specifically, we’ll order v1, v2, and v3 such that (v2 – v1) × (v3 – v1) points outside the octahedron. If an outward-pointing vector is aimed toward us, it means the face is visible from our perspective. Otherwise, the face is obscured and we won’t need to draw it.

<img src='https://github.com/rahiakela/img-repo/blob/master/math-for-programmers/octahedron-face.png?raw=1' width='800'/>

We can define the eight triangular faces as triples of three vectors v1, v2, and v3 as follows:







In [3]:
octahedron = [
  [(1,0,0), (0,1,0), (0,0,1)],
  [(1,0,0), (0,0,-1), (0,1,0)],
  [(1,0,0), (0,0,1), (0,-1,0)],
  [(1,0,0), (0,-1,0), (0,0,-1)],
  [(-1,0,0), (0,0,1), (0,1,0)],
  [(-1,0,0), (0,1,0), (0,0,-1)],
  [(-1,0,0), (0,-1,0), (0,0,1)],
  [(-1,0,0), (0,0,-1), (0,-1,0)],            
]

The faces are actually the only data we need to render the shape; these contain the edges and vertices implicitly.

In [4]:
def vertices(faces):
  return list(set([vertex for face in faces for vertex in face]))

## Projecting to 2D

To turn 3D points into 2D points, we must choose what 3D direction we are observing from. Once we have two 3D vectors defining “up” and “right” from our perspective, we can project any 3D vector onto them and get two components instead of three.

Let's extracts the part of any 3D vector pointing in a given direction
using the dot product:

In [5]:
def dot(u, v):
  return sum([coord1 * coord2 for coord1, coord2 in zip(u, v)])

def component(v, direction):
  return (dot(v, direction) / length(direction))

With two directions, we can establish a way to project from three coordinates down to two. This function takes a 3D vector or a tuple of three numbers and returns a 2D vector or a tuple of two numbers:

In [6]:
def vector_to_2d(v):
  return (component(v, (1, 0, 0)), component(v, (0, 1, 0)))

We can picture this as “flattening” the 3D vector into the plane. Deleting the z component takes away any depth the vector has.

<img src='https://github.com/rahiakela/img-repo/blob/master/math-for-programmers/3d-flattens.png?raw=1' width='800'/>

Finally, to take a triangle from 3D to 2D, we need only apply this function to all of the vertices defining a face:

In [7]:
def face_to_2d(face):
  return [vector_to_2d(vertex) for vertex in face]

## Orienting faces and shading

As a positive number, this result confirms that the vectors are separated by less than 90°. These vectors have the same relative geometry whether we consider them in 2D or in 3D as the vectors (2, 3, 0) and (4, 5, 0) that happen to lie in the plane where z = 0.

<img src='https://github.com/rahiakela/img-repo/blob/master/math-for-programmers/dot-product-3d.png?raw=1' width='800'/>

In [ ]:
u = [2, 3, 0]
v = [4, 5, 0]
d = dot(u, v)
print(d)

23


## Dot products by example

It’s not surprising that two vectors lying on different axes have zero dot product. We know they are perpendicular:

In [ ]:
dot((1, 0), (0, 2))

0

In [ ]:
dot((0, 3, 0), (0, 0, -5))

0

We can also confirm that longer vectors give longer dot products.

For instance, scaling either input vector by a factor of 2 doubles the output of the dot product:

In [ ]:
dot((3, 4), (2, 3))

18

In [ ]:
dot(scale(2, (3, 4)), (2, 3))

36

In [ ]:
dot((3, 4), scale(2, (2, 3)))

36

It turns out the dot product is proportional to each of the lengths of its input vectors. If you take the dot product of two vectors in the same direction, the dot product is precisely equal to the product of the lengths. 

For instance, (4, 3) has a length of 5 and (8, 6) has a length of 10. The dot product is equal to 5 · 10:

In [ ]:
dot((4, 3), (8, 6))

50

Of course, the dot product is not always equal to the product of the lengths of its inputs. The vectors (5, 0), (–3, 4), (0, –5), and (–4, –3) all have the same length of 5 but different dot products with the original vector (4, 3).

<img src='https://github.com/rahiakela/img-repo/blob/master/math-for-programmers/different-dot-products.png?raw=1' width='800'/>

In [ ]:
dot((4, 3), (4, 3))

25

In [ ]:
dot((-3, 4), (4, 3))

0

In [ ]:
dot((4, 3), (-4, -3))

-25

In [ ]:
dot((4, 3), (0, -5))

-15

In [ ]:
dot((4, 3), (5, 0))

20

The dot product of two vectors of length 5 ranges from 5 · 5 = 25 when they are
aligned to –25, when they point in opposite directions.

## Measuring angles with the dot product

We’ve seen that the dot product varies based on the angle between two vectors. Specifically, the dot product $u · v$ ranges from 1 to –1 times the product of the lengths of $u$ and $v$ as the angle ranges from 0 to $180°$.


If |u| and |v| denote the lengths of vectors u and v, the dot product is given by

$$u . v = \|u\| . \|v\| . cos(\theta)$$

where $\theta$ is the angle between the vectors u and v. In principle this gives us a new way to compute a dot product. We could measure the lengths of two vectors and then measure the angle between them to get the result. Suppose, we have two vectors of known lengths 3 and 2, respectively, and using our protractor, discovered that they are 75° apart.

<img src='https://github.com/rahiakela/img-repo/blob/master/math-for-programmers/two-vectors-length.png?raw=1' width='800'/>

The dot product for the two vectors is 3 · 2 · cos(75°). With the appropriate
conversion to radians, we can compute this in Python to be about 1.55:

In [ ]:
from math import cos, pi, acos

In [ ]:
3 * 2 * cos(75 * pi / 180)

1.5529142706151244

When doing computations with vectors, it’s more common to start with coordinates
and to compute angles from them. We can combine both of our formulas to recover
an angle: first we compute the dot product and lengths using coordinates, then we solve for the angle.

Let’s find the angle between the vectors (3, 4) and (4, 3). Their dot product is 24, and each of their lengths is 5. Our new dot product formula tells us that:

$(3, 4) · (4, 3) = 24 = 5 · 5 · cos(\theta) = 25 · cos(\theta)$

In [ ]:
dot((3, 4), (4, 3))

24

In [ ]:
theta = 24 / 25
theta

0.96

In [ ]:
acos(theta)

0.283794109208328

From $24 = 25 · cos(\theta)$, we can simplify it to $cos(\theta) = 24/25$. Using Python’s `math.acos`, we find that a $\theta$ value of 0.284 radians or 16.3° gives us a cosine of 24/25.

**This exercise reminds us why we don’t need the dot product in 2D.**

Using that formula creatively, we could find any angle we want in the plane. The dot product really starts to shine in 3D, where a change of coordinates can’t help us as much.

For instance, we can use the same formula to find the angle between (1, 2, 2) and (2, 2, 1). The dot product is 1 · 2 + 2 · 2 + 2 · 1 = 8 and the lengths are both 3. This means $8 = 3 · 3 · cos(\theta)$, so $cos(\theta) = 8/9$ and $\theta = 0.476$ radians or 27.3°.


This process is the same in 2D or 3D, and it’s one we’ll use over and over. We can save some effort by implementing a Python function to find the angle between two vectors. Because neither our dot function nor our length function has a hard-coded number of dimensions, this new function won’t either. We can make use of the fact that $u . v = \|u\| . \|v\| . cos(\theta)$ and, therefore

$$ cos(\theta) = \frac{u . v}  {\|u\| . \|v\|}$$

And


$$ \theta = arccos \left( \frac{u . v}  {\|u\| . \|v\|} \right)$$

This formula translates neatly to Python code as follows:




In [ ]:
def angle_between(v1, v2):
  return acos(dot(v1, v2) / (length(v1) * length(v2)))

## Exercises

**Ex-3.12**: What is the dot product of (–1, –1, 1) and (1, 2, 1)? Are these
two 3D vectors separated by more than 90°, less than 90°, or exactly 90°?

In [ ]:
dot((-1, -1, 1), (1, 2, 1))

-2

Because this is a negative number, the two vectors are more than 90° apart.

**Ex-3.15**: Find a vector u of length 3 and a vector v of length
7 such that u · v = 21. Find another pair of vectors u and v such that u · v = –21.
Finally, find three more pairs of vectors of respective lengths 3 and 7 and show
that all of their lengths lie between –21 and 21.


**Solution** Two vectors in the same direction (for instance, along the positive
x-axis) will have the highest possible dot product:


In [ ]:
dot((3, 0), (7, 0))

21

Two vectors in the opposite direction (for instance, the positive and negative y
directions) will have the lowest possible dot product:

In [ ]:
dot((0, 3), (0, -7))

-21

Using polar coordinates, we can easily generate some more vectors of length 3
and 7 with random angles:

In [ ]:
from vectors import to_cartesian
from random import random
from math import pi

In [ ]:
def random_vector_of_length(l):
  return to_cartesian((l, 2 * pi * random()))

pairs = [(random_vector_of_length(3), random_vector_of_length(7)) for i in range(0, 3)]

for u, v in pairs:
  print("u = %s, v = %s" % (u, v))
  print("length of u: %f, length of v: %f, dot product :%f" % (length(u), length(v), dot(u, v)))

u = (0.1607267582104504, -2.995691390847054), v = (4.24671801343339, -5.564655075957571)
length of u: 3.000000, length of v: 7.000000, dot product :17.352551
u = (-2.725245914585489, -1.2542068031369873), v = (6.155983372194031, -3.3322467977733288)
length of u: 3.000000, length of v: 7.000000, dot product :-12.597242
u = (-2.729303533959927, 1.2453522471629674), v = (2.9246714434742693, -6.359740320777739)
length of u: 3.000000, length of v: 7.000000, dot product :-15.902433


**Ex-3.16** Let u and v be vectors, with |u| = 3.61 and |v| = 1.44. If the angle
between u and v is 101.3°, what is u · v?

**Solution**

Again, we can plug these values into the new dot product formula
and, with the appropriate conversion to radians, evaluate the result in Python:

In [ ]:
3.61 * 1.44 * cos(101.3 * pi / 180)

-1.0186064362303022

**Ex-3.17**: Find the angle between (3, 4) and (4, 3) by converting
them to polar coordinates and taking the difference of the angles.

**Hint** The result should agree with the value from the dot product formula.

**Solution** The vector (3, 4) is further from the positive x-axis counterclockwise, so we subtract the angle of (4, 3) from the angle of (3, 4) to get our answer.

In [ ]:
from vectors import to_polar

In [ ]:
r1, t1 = to_polar((4, 3))
r2, t2 = to_polar((3, 4))

t1 - t2

-0.2837941092083278

In [ ]:
t2 - t1

0.2837941092083278

**Ex-3.18**: What is the angle between (1, 1, 1) and (–1, –1, 1) in degrees?


In [ ]:
length((1, 1, 1))

1.7320508075688772

In [ ]:
length((-1, -1, 1))

1.7320508075688772

The lengths of both vectors are $\sqrt{3}$ or approximately 1.732.

In [ ]:
dot((1, 1, 1,), (-1, -1, 1))

-1

so $–1 = \sqrt{3} . \sqrt{3}. cos(\theta)$. Therefore, $cos(\theta) = –1/3$. This makes the angle approximately 1.911 radians or 109.5°